In [1]:
import numpy as np
import cv2
import os
from scipy import ndimage
from scipy.ndimage import convolve
from scipy import misc
import pandas as pd

**Preprocessing**

In [2]:
sigma=1.4
kernel_size=5
lowthreshold=0.09
highthreshold=0.17
strong_pixel=255
weak_pixel=100
img_smoothed = None
gradientMat = None
thetaMat = None
nonMaxImg = None
thresholdImg = None

def rgb2gray(rgb):
    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b
    return gray

def gaussian_kernel(size, sigma=1):
    size = int(size) // 2
    x, y = np.mgrid[-size:size+1, -size:size+1]
    normal = 1 / (2.0 * np.pi * sigma**2)
    g =  np.exp(-((x**2 + y**2) / (2.0*sigma**2))) * normal
    return g

def sobel_filters(img):
    Kx = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], np.float32)
    Ky = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]], np.float32)
    Ix = ndimage.convolve(img, Kx)
    Iy = ndimage.convolve(img, Ky)
    G = np.hypot(Ix, Iy)
    G = G / G.max() * 255
    theta = np.arctan2(Iy, Ix)
    return (G, theta)

def non_max_suppression(img, D):
    M, N = img.shape
    Z = np.zeros((M,N), dtype=np.int32)
    angle = D * 180. / np.pi
    angle[angle < 0] += 180
    for i in range(1,M-1):
        for j in range(1,N-1):
            try:
                q = 255
                r = 255
                if (0 <= angle[i,j] < 22.5) or (157.5 <= angle[i,j] <= 180):
                    q = img[i, j+1]
                    r = img[i, j-1]
                elif (22.5 <= angle[i,j] < 67.5):
                    q = img[i+1, j-1]
                    r = img[i-1, j+1]
                elif (67.5 <= angle[i,j] < 112.5):
                    q = img[i+1, j]
                    r = img[i-1, j]
                elif (112.5 <= angle[i,j] < 157.5):
                    q = img[i-1, j-1]
                    r = img[i+1, j+1]
                if (img[i,j] >= q) and (img[i,j] >= r):
                    Z[i,j] = img[i,j]
                else:
                    Z[i,j] = 0
            except IndexError as e:
                pass
    return Z

def threshold(img):
    global highthreshold
    global lowthreshold
    global strong_pixel
    global weak_pixel
    highThresholdP = img.max() * highthreshold;
    lowThresholdP = highThresholdP * lowthreshold;
    M, N = img.shape
    res = np.zeros((M,N), dtype=np.int32)
    weak = np.int32(weak_pixel)
    strong = np.int32(strong_pixel)
    strong_i, strong_j = np.where(img >= highThresholdP)
    zeros_i, zeros_j = np.where(img < lowThresholdP)
    weak_i, weak_j = np.where((img <= highThresholdP) & (img >= lowThresholdP))
    res[strong_i, strong_j] = strong
    res[weak_i, weak_j] = weak
    return (res)

def hysteresis(img):
    global strong_pixel
    global weak_pixel
    M, N = img.shape
    weak = weak_pixel
    strong = strong_pixel
    for i in range(1, M-1):
        for j in range(1, N-1):
            if (img[i,j] == weak):
                try:
                    if ((img[i+1, j-1] == strong) or (img[i+1, j] == strong) or (img[i+1, j+1] == strong)
                        or (img[i, j-1] == strong) or (img[i, j+1] == strong)
                        or (img[i-1, j-1] == strong) or (img[i-1, j] == strong) or (img[i-1, j+1] == strong)):
                        img[i, j] = strong
                    else:
                        img[i, j] = 0
                except IndexError as e:
                    pass
    return img

def canny_preprocessing(img):
    img_smoothed = convolve(img, gaussian_kernel(kernel_size, sigma))
    gradientMat, thetaMat = sobel_filters(img_smoothed)
    nonMaxImg = non_max_suppression(gradientMat, thetaMat)
    thresholdImg = threshold(nonMaxImg)
    img_final = hysteresis(thresholdImg)
    return img_final


In [3]:
classified_image=["train","test"]
categories=["0","1","2","3","4","5","6","7","8","9","A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z"]
category_index={"0":0,"1":1,"2":2,"3":3,"4":4,"5":5,"6":6,"7":7,"8":8,"9":9,"A":10,"B":11,"C":12,"D":13,"E":14,"F":15,"G":16,"H":17,"I":18,"J":19,"K":20,"L":21,"M":22,"N":23,"O":24,"P":25,"Q":26,"R":27,"S":28,"T":29,"U":30,"V":31,"W":32,"X":33,"Y":34,"Z":35}

x_train=[]    #the values of trained images are stored in this array
y_train=[]    #the indexes of trained images that are beloged to their category are stored in this array
x_test=[]     #the values of tested images are stored in this array
y_test=[]     #the indexes of that tested images that are beloged to their category are stored in this array

Change the paths accordingly and run the below cell

In [4]:
data_path="/Users/dhatrirukkannagari/Desktop/HackUTA/data1"

for classify_img in classified_image:
    
    path1=os.path.join(data_path,classify_img)   
    print("Extracting the "+classify_img+" images/.......")
    
    for category in categories:
        folder_path=os.path.join(path1,category) 
        images=os.listdir(folder_path)
        
        print(f'    loading images of the category {category} in {classify_img} images.......')
        
        for image in images:
            img_path=os.path.join(folder_path,image)
            try:
                img=cv2.imread(img_path)
                img=rgb2gray(img)
                preprocessed_img=canny_preprocessing(img)
                if classify_img=="train":
                    x_train.append(preprocessed_img)
                    y_train.append(category_index[category])
                else:
                    x_test.append(preprocessed_img)
                    y_test.append(category_index[category])
                    
            
            except:
                pass

print("Fetching data completed........")

Extracting the train images/.......
    loading images of the category 0 in train images.......
    loading images of the category 1 in train images.......
    loading images of the category 2 in train images.......
    loading images of the category 3 in train images.......
    loading images of the category 4 in train images.......
    loading images of the category 5 in train images.......
    loading images of the category 6 in train images.......
    loading images of the category 7 in train images.......
    loading images of the category 8 in train images.......
    loading images of the category 9 in train images.......
    loading images of the category A in train images.......
    loading images of the category B in train images.......
    loading images of the category C in train images.......
    loading images of the category D in train images.......
    loading images of the category E in train images.......
    loading images of the category F in train images.......
    

In [5]:
x_train=np.array(x_train)
y_train=np.array(y_train)
x_test=np.array(x_test)
y_test=np.array(y_test)

In [6]:
x_train.shape

(1801, 261, 310)

In [7]:
x_test.shape

(720, 261, 310)

In [8]:
y_train.shape

(1801,)

In [9]:
y_test.shape

(720,)

In [10]:
#Althogh when we converted to grayscale the imgs still have 3 color channels(BGR),so we have to RESHAPE it
x_train=np.reshape(x_train,(x_train.shape[0],261,310,1))
x_test=np.reshape(x_test,(x_test.shape[0],261,310,1))

In [11]:
#there are 36 categories so..
numOfCategories=36

In [13]:
from keras.utils import to_categorical
y_train=to_categorical(y_train,numOfCategories)
y_test=to_categorical(y_test,numOfCategories)

In [18]:
import tensorflow as tf

In [19]:
import keras

In [20]:

from keras.models import Sequential
from keras.layers import Dense,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.layers import BatchNormalization

In [21]:
model=Sequential()

model.add(Conv2D(64,kernel_size=(3,3),input_shape=(261, 310, 1),activation="relu"))
model.add(BatchNormalization())
model.add(Conv2D(64,kernel_size=(3,3),input_shape=(261, 310, 1),activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(128,kernel_size=(3,3),activation="relu"))
model.add(BatchNormalization())
model.add(Conv2D(128,kernel_size=(3,3),activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256,kernel_size=(3,3),activation="relu"))
model.add(BatchNormalization())
model.add(Conv2D(256,kernel_size=(3,3),activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512,kernel_size=(3,3),activation="relu"))
model.add(BatchNormalization())
model.add(Conv2D(512,kernel_size=(3,3),activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.4))



model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(128,activation="relu"))
model.add(Dense(64,activation="relu"))
model.add(Dense(36,activation="softmax"))

In [22]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 259, 308, 64)      640       
                                                                 
 batch_normalization (Batch  (None, 259, 308, 64)      256       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 257, 306, 64)      36928     
                                                                 
 batch_normalization_1 (Bat  (None, 257, 306, 64)      256       
 chNormalization)                                                
                                                                 
 max_pooling2d (MaxPooling2  (None, 128, 153, 64)      0         
 D)                                                              
                                                      

In [24]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [26]:
model.fit(x_train,y_train,batch_size=50,epochs=10,validation_data=(x_test,y_test))


Epoch 1/10
37/37 [==============================] - 511s 14s/step - loss: 4.3669 - accuracy: 0.3054 - val_loss: 175.6246 - val_accuracy: 0.0569
Epoch 2/10
37/37 [==============================] - 509s 14s/step - loss: 0.8405 - accuracy: 0.7551 - val_loss: 41.4877 - val_accuracy: 0.1083
Epoch 3/10
37/37 [==============================] - 515s 14s/step - loss: 0.2775 - accuracy: 0.9323 - val_loss: 14.0850 - val_accuracy: 0.1847
Epoch 4/10
37/37 [==============================] - 507s 14s/step - loss: 0.1411 - accuracy: 0.9634 - val_loss: 8.7234 - val_accuracy: 0.2722
Epoch 5/10
37/37 [==============================] - 1230s 34s/step - loss: 0.5118 - accuracy: 0.8717 - val_loss: 29.6007 - val_accuracy: 0.1847
Epoch 6/10
37/37 [==============================] - 497s 13s/step - loss: 0.2195 - accuracy: 0.9445 - val_loss: 16.3940 - val_accuracy: 0.2236
Epoch 7/10
37/37 [==============================] - 500s 14s/step - loss: 0.0415 - accuracy: 0.9872 - val_loss: 11.5884 - val_accuracy: 0.305

In [28]:
#Saving the model
model_json=model.to_json()
with open("isl_model.json","w") as json_file:
    json_file.write(model_json)
    #serializing weights to HDF5
model.save_weights("isl_model.h5")